In [1]:
def ad_fuller(col_values):
    result = adfuller(col_values)
    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))

In [2]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA

#visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# for imputation
from pyod.models.knn  import KNN
import pyod

#model building
from sklearn.model_selection import train_test_split,KFold,cross_val_score,StratifiedKFold,GridSearchCV
from sklearn.metrics import mean_squared_log_error
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LinearRegression,ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import train_test_split,KFold,GridSearchCV,cross_val_score
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import lightgbm 
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier


pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

import warnings
warnings.filterwarnings('ignore')

import gc
gc.collect()

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [99]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train['Course_ID'] = train['Course_ID'].astype('object')
train['Day_No'] = train['Day_No'].astype('object')

In [100]:
train.head()

,ID,Day_No,Course_ID,Course_Domain,Course_Type,Short_Promotion,Public_Holiday,Long_Promotion,User_Traffic,Competition_Metric,Sales
0,1,1,1,Development,Course,0,1,1,11004,0.007,81
1,2,2,1,Development,Course,0,0,1,13650,0.007,79
2,3,3,1,Development,Course,0,0,1,11655,0.007,75
3,4,4,1,Development,Course,0,0,1,12054,0.007,80
4,5,5,1,Development,Course,0,0,1,6804,0.007,41


In [101]:
test.head()

,ID,Day_No,Course_ID,Course_Domain,Course_Type,Short_Promotion,Public_Holiday,Long_Promotion,Competition_Metric
0,883,883,1,Development,Course,1,0,1,0.007
1,884,884,1,Development,Course,1,0,1,0.007
2,885,885,1,Development,Course,1,0,1,0.007
3,886,886,1,Development,Course,1,0,1,0.007
4,887,887,1,Development,Course,0,0,1,0.007


In [6]:
train['Competition_Metric'].fillna(train.Competition_Metric.mean(),inplace=True)
train['Competition_Metric'] = np.exp(train['Competition_Metric'])

test['Competition_Metric'].fillna(test.Competition_Metric.mean(),inplace=True)
test['Competition_Metric'] = np.exp(test['Competition_Metric'])

In [7]:
cols =['ID','Day_No','Course_ID','Course_Domain','Course_Type']
data = pd.concat([train[cols],test[cols]])
data.index = np.arange(0,len(data))
data.head()

,ID,Day_No,Course_ID,Course_Domain,Course_Type
0,1,1,1,Development,Course
1,2,2,1,Development,Course
2,3,3,1,Development,Course
3,4,4,1,Development,Course
4,5,5,1,Development,Course


In [8]:
arima_df = pd.DataFrame(columns=['Day_No'])
arima_df['Day_No']=np.arange(1,data.Day_No.max()+1)

df = train.groupby(['Day_No']).agg({\
                                   'User_Traffic':{\
                                           "user_traffic_mean":'mean',
                                            "user_traffic_std":'std'
                                           },
                                   }).reset_index()
df.columns= ['Day_No','user_traffic_mean','user_traffic_std']
df.head()

model = ARIMA(df.user_traffic_mean,order=(3,0,3))
results_arima = model.fit()
pred=results_arima.predict(start=0,end=data.Day_No.max()+1)
arima_df['day_user_traffic_mean_arima']=round(pred,0)
#print("rmse",np.sqrt(sum((df.user_traffic_mean-results_arima.fittedvalues)**2)))
#for user_traffic mean arima==(3,0,0)

model = ARIMA(df.user_traffic_std,order=(2,0,0))
results_arima = model.fit()
pred=results_arima.predict(start=0,end=data.Day_No.max()+1)
arima_df['day_user_traffic_std_arima']=round(pred,0)
#print("rmse",np.sqrt(sum((df.user_traffic_std-results_arima.fittedvalues)**2)))
#for user_traffic mean arima==(2,0,0)
arima_df.head()

arima_df['Day_No']=arima_df['Day_No'].astype(int)

data = pd.merge(data,arima_df,on=['Day_No'],how='left')
data.head()

,ID,Day_No,Course_ID,Course_Domain,Course_Type,day_user_traffic_mean_arima,day_user_traffic_std_arima
0,1,1,1,Development,Course,15398.0,7242.0
1,2,2,1,Development,Course,15460.0,7258.0
2,3,3,1,Development,Course,15454.0,7333.0
3,4,4,1,Development,Course,15152.0,7229.0
4,5,5,1,Development,Course,15261.0,6956.0


In [9]:
df = train.groupby(['Day_No','Course_Domain']).agg({\
                                   'User_Traffic':{\
                                           "user_traffic_mean":'mean',
                                            "user_traffic_std":'std'
                                           }
                                   }).reset_index().fillna(0)
df.columns= ['Day_No','Course_Domain','user_traffic_mean','user_traffic_std']

temp_df = data.copy()
temp_df = temp_df.groupby(["Day_No",'Course_Domain']).size().reset_index()
temp_df.columns=["Day_No",'Course_Domain','remove']
temp_df.drop(['remove'],axis=1,inplace=True)


vals = list(temp_df.Course_Domain.unique())
for val in vals:
    temp = pd.DataFrame(columns=['Day_No'])
    temp['Day_No']=np.arange(1,data.Day_No.max()+1)
    temp['Course_Domain']=val
    
    model = ARIMA(df[df["Course_Domain"]==val].user_traffic_mean,order=(3,0,0))
    results_arima = model.fit()
    pred=results_arima.predict(start=0,end=data.Day_No.max())
    temp["user_traffic_dom_mean_arima_"+str(val)]=round(pred,0)
    
    model = ARIMA(df[df["Course_Domain"]==val].user_traffic_std,order=(3,0,0))
    results_arima = model.fit()
    pred=results_arima.predict(start=0,end=data.Day_No.max())
    temp["user_traffic_dom_std_arima_"+str(val)]=round(pred,0)
    
    temp_df = pd.merge(temp_df,temp,on=['Day_No','Course_Domain'],how='outer')
    
temp_df['user_traffic_dom_mean_arima_Business'].fillna(temp_df['user_traffic_dom_mean_arima_Development'],inplace=True)
temp_df['user_traffic_dom_mean_arima_Business'].fillna(temp_df['user_traffic_dom_mean_arima_Finance & Accounting'],inplace=True)
temp_df['user_traffic_dom_mean_arima_Business'].fillna(temp_df['user_traffic_dom_mean_arima_Software Marketing'],inplace=True)

temp_df['user_traffic_dom_std_arima_Business'].fillna(temp_df['user_traffic_dom_std_arima_Development'],inplace=True)
temp_df['user_traffic_dom_std_arima_Business'].fillna(temp_df['user_traffic_dom_std_arima_Finance & Accounting'],inplace=True)
temp_df['user_traffic_dom_std_arima_Business'].fillna(temp_df['user_traffic_dom_std_arima_Software Marketing'],inplace=True)

temp_df.drop(['user_traffic_dom_mean_arima_Development','user_traffic_dom_mean_arima_Finance & Accounting','user_traffic_dom_mean_arima_Software Marketing',\
        'user_traffic_dom_std_arima_Development','user_traffic_dom_std_arima_Finance & Accounting','user_traffic_dom_std_arima_Software Marketing'],\
       axis=1,inplace=True)

temp_df.rename(columns={"user_traffic_dom_mean_arima_Business":'user_traffic_dom_mean_arima',\
                  "user_traffic_dom_std_arima_Business":"user_traffic_dom_std_arima"},inplace=True)
temp_df.head()

# train = pd.merge(train,temp_df[["Day_No","Course_Domain","user_traffic_dom_mean_arima","user_traffic_dom_std_arima"]],on=['Day_No','Course_Domain'],how='left')
# test = pd.merge(test,temp_df[["Day_No","Course_Domain","user_traffic_dom_mean_arima","user_traffic_dom_std_arima"]],on=['Day_No','Course_Domain'],how='left')
data = pd.merge(data,temp_df[["Day_No","Course_Domain","user_traffic_dom_mean_arima","user_traffic_dom_std_arima"]],on=['Day_No','Course_Domain'],how='left')
data.head()

,ID,Day_No,Course_ID,Course_Domain,Course_Type,day_user_traffic_mean_arima,day_user_traffic_std_arima,user_traffic_dom_mean_arima,user_traffic_dom_std_arima
0,1,1,1,Development,Course,15398.0,7242.0,16214.0,7333.0
1,2,2,1,Development,Course,15460.0,7258.0,16098.0,7389.0
2,3,3,1,Development,Course,15454.0,7333.0,16370.0,7452.0
3,4,4,1,Development,Course,15152.0,7229.0,15934.0,7381.0
4,5,5,1,Development,Course,15261.0,6956.0,16144.0,7149.0


In [10]:
df = train.groupby(['Day_No','Course_Type']).agg({\
                                   'User_Traffic':{\
                                           "user_traffic_mean":'mean',
                                            "user_traffic_std":'std'
                                           }
                                   }).reset_index().fillna(0)
df.columns= ['Day_No','Course_Type','user_traffic_mean','user_traffic_std']
df.head()

temp_df = data.copy()
temp_df = temp_df.groupby(["Day_No",'Course_Type']).size().reset_index()
temp_df.columns=["Day_No",'Course_Type','remove']
temp_df.drop(['remove'],axis=1,inplace=True)

vals = list(df.Course_Type.unique())
for val in vals:
    temp = pd.DataFrame(columns=['Day_No'])
    temp['Day_No']=np.arange(1,data.Day_No.max()+1)
    temp['Course_Type']=val
    
    model = ARIMA(df[df["Course_Type"]==val].user_traffic_mean,order=(3,0,0))
    results_arima = model.fit()
    pred=results_arima.predict(start=0,end=data.Day_No.max())
    temp["user_traffic_ctype_mean_arima_"+str(val)]=round(pred,0)
    
    model = ARIMA(df[df["Course_Type"]==val].user_traffic_std,order=(3,0,0))
    results_arima = model.fit()
    pred=results_arima.predict(start=0,end=data.Day_No.max())
    temp["user_traffic_ctype_std_arima_"+str(val)]=round(pred,0)
    
    temp_df = pd.merge(temp_df,temp,on=['Day_No','Course_Type'],how='left')

    
temp_df['user_traffic_ctype_mean_arima_Course'].fillna(temp_df['user_traffic_ctype_mean_arima_Degree'],inplace=True)
temp_df['user_traffic_ctype_mean_arima_Course'].fillna(temp_df['user_traffic_ctype_mean_arima_Program'],inplace=True)

temp_df['user_traffic_ctype_std_arima_Course'].fillna(temp_df['user_traffic_ctype_std_arima_Degree'],inplace=True)
temp_df['user_traffic_ctype_std_arima_Course'].fillna(temp_df['user_traffic_ctype_std_arima_Program'],inplace=True)

temp_df.drop(['user_traffic_ctype_mean_arima_Degree','user_traffic_ctype_mean_arima_Program','user_traffic_ctype_std_arima_Degree',\
        'user_traffic_ctype_std_arima_Program'],\
       axis=1,inplace=True)

temp_df.rename(columns={"user_traffic_ctype_mean_arima_Course":'user_traffic_ctype_mean_arima',\
                  "user_traffic_ctype_std_arima_Course":"user_traffic_ctype_std_arima"},inplace=True)
temp_df.drop_duplicates(keep='first',inplace=True)
temp_df.head()

# # train = pd.merge(train,temp_df[["Day_No","Course_Type","user_traffic_ctype_mean_arima","user_traffic_ctype_std_arima"]],on=['Day_No','Course_Type'],how='left')
# # test = pd.merge(test,temp_df[["Day_No","Course_Type","user_traffic_ctype_mean_arima","user_traffic_ctype_std_arima"]],on=['Day_No','Course_Type'],how='left')
data = pd.merge(data,temp_df[["Day_No","Course_Type","user_traffic_ctype_mean_arima","user_traffic_ctype_std_arima"]],on=['Day_No','Course_Type'],how='left')
data.head()

,ID,Day_No,Course_ID,Course_Domain,Course_Type,day_user_traffic_mean_arima,day_user_traffic_std_arima,user_traffic_dom_mean_arima,user_traffic_dom_std_arima,user_traffic_ctype_mean_arima,user_traffic_ctype_std_arima
0,1,1,1,Development,Course,15398.0,7242.0,16214.0,7333.0,14990.0,6415.0
1,2,2,1,Development,Course,15460.0,7258.0,16098.0,7389.0,15125.0,6475.0
2,3,3,1,Development,Course,15454.0,7333.0,16370.0,7452.0,15170.0,6557.0
3,4,4,1,Development,Course,15152.0,7229.0,15934.0,7381.0,14955.0,6495.0
4,5,5,1,Development,Course,15261.0,6956.0,16144.0,7149.0,14958.0,6270.0


In [11]:
# model = ARIMA(df[df["Course_ID"]==1].user_traffic_mean,order=(3,0,2))
# results_arima = model.fit()
# print("rmse",np.sqrt(sum((df[df["Course_ID"]==1].user_traffic_mean-results_arima.fittedvalues)**2)))
# p=3,q=2 but for some svd wont converge so q=0 or 1
df = train.groupby(['Day_No','Course_ID']).agg({\
                                   'User_Traffic':{\
                                           "user_traffic_mean":'mean',
                                           }
                                   }).reset_index().fillna(0)
df.columns= ['Day_No','Course_ID','user_traffic_mean']


temp = pd.DataFrame(columns=['Day_No'])
temp['Day_No']=np.arange(1,data.Day_No.max()+1)

vals = list(df.Course_ID.unique())
for val in vals:
    temp[str(val)]=np.nan
    
    model = ARIMA(df[df["Course_ID"]==val].user_traffic_mean,order=(2,0,0))
    results_arima = model.fit()
    pred=results_arima.predict(start=0,end=test.Day_No.max())
    temp[str(val)]=round(pred,0)
    
t = temp.melt(id_vars=["Day_No"], 
              var_name="Course_ID",\
              value_name="user_traffic_id_arima")
t['Day_No'] = t['Day_No'] .astype(int)
t['Course_ID'] = t['Course_ID'].astype(int)
# train = pd.merge(train,t,on=['Day_No','Course_ID'],how='left')
# test = pd.merge(test,t,on=['Day_No','Course_ID'],how='left')
data = pd.merge(data,t,on=['Day_No','Course_ID'],how='left')
data.head()

,ID,Day_No,Course_ID,Course_Domain,Course_Type,day_user_traffic_mean_arima,day_user_traffic_std_arima,user_traffic_dom_mean_arima,user_traffic_dom_std_arima,user_traffic_ctype_mean_arima,user_traffic_ctype_std_arima,user_traffic_id_arima
0,1,1,1,Development,Course,15398.0,7242.0,16214.0,7333.0,14990.0,6415.0,11882.0
1,2,2,1,Development,Course,15460.0,7258.0,16098.0,7389.0,15125.0,6475.0,11617.0
2,3,3,1,Development,Course,15454.0,7333.0,16370.0,7452.0,15170.0,6557.0,12428.0
3,4,4,1,Development,Course,15152.0,7229.0,15934.0,7381.0,14955.0,6495.0,11798.0
4,5,5,1,Development,Course,15261.0,6956.0,16144.0,7149.0,14958.0,6270.0,11936.0


In [12]:
data.columns

Index(['ID', 'Day_No', 'Course_ID', 'Course_Domain', 'Course_Type',
       'day_user_traffic_mean_arima', 'day_user_traffic_std_arima',
       'user_traffic_dom_mean_arima', 'user_traffic_dom_std_arima',
       'user_traffic_ctype_mean_arima', 'user_traffic_ctype_std_arima',
       'user_traffic_id_arima'],
      dtype='object')

In [13]:
cols = ['ID', 
       'day_user_traffic_mean_arima', 'day_user_traffic_std_arima',
       'user_traffic_dom_mean_arima', 'user_traffic_dom_std_arima',
       'user_traffic_ctype_mean_arima', 'user_traffic_ctype_std_arima',
       'user_traffic_id_arima']
data = data[cols]
data.head()

train=pd.merge(train,data,on='ID',how='left')
train.drop(['User_Traffic'],axis=1,inplace=True)
test=pd.merge(test,data,on='ID',how='left')

In [14]:
train.head()

,ID,Day_No,Course_ID,Course_Domain,Course_Type,Short_Promotion,Public_Holiday,Long_Promotion,Competition_Metric,Sales,day_user_traffic_mean_arima,day_user_traffic_std_arima,user_traffic_dom_mean_arima,user_traffic_dom_std_arima,user_traffic_ctype_mean_arima,user_traffic_ctype_std_arima,user_traffic_id_arima
0,1,1,1,Development,Course,0,1,1,1.007025,81,15398.0,7242.0,16214.0,7333.0,14990.0,6415.0,11882.0
1,2,2,1,Development,Course,0,0,1,1.007025,79,15460.0,7258.0,16098.0,7389.0,15125.0,6475.0,11617.0
2,3,3,1,Development,Course,0,0,1,1.007025,75,15454.0,7333.0,16370.0,7452.0,15170.0,6557.0,12428.0
3,4,4,1,Development,Course,0,0,1,1.007025,80,15152.0,7229.0,15934.0,7381.0,14955.0,6495.0,11798.0
4,5,5,1,Development,Course,0,0,1,1.007025,41,15261.0,6956.0,16144.0,7149.0,14958.0,6270.0,11936.0


In [15]:
test.head()

,ID,Day_No,Course_ID,Course_Domain,Course_Type,Short_Promotion,Public_Holiday,Long_Promotion,Competition_Metric,day_user_traffic_mean_arima,day_user_traffic_std_arima,user_traffic_dom_mean_arima,user_traffic_dom_std_arima,user_traffic_ctype_mean_arima,user_traffic_ctype_std_arima,user_traffic_id_arima
0,883,883,1,Development,Course,1,0,1,1.007025,16577.0,7470.0,17016.0,7637.0,15449.0,6614.0,12577.0
1,884,884,1,Development,Course,1,0,1,1.007025,14811.0,7708.0,15911.0,7746.0,14797.0,6729.0,12075.0
2,885,885,1,Development,Course,1,0,1,1.007025,15885.0,7415.0,16921.0,7662.0,15718.0,6656.0,11935.0
3,886,886,1,Development,Course,1,0,1,1.007025,16763.0,7412.0,16502.0,7521.0,15211.0,6559.0,11897.0
4,887,887,1,Development,Course,0,0,1,1.007025,15008.0,7330.0,16230.0,7492.0,15035.0,6536.0,11886.0


In [16]:
cols =['ID','Day_No','Course_ID','Course_Domain','Course_Type']
data = pd.concat([train[cols],test[cols]])
data.index = np.arange(0,len(data))
data.head()

,ID,Day_No,Course_ID,Course_Domain,Course_Type
0,1,1,1,Development,Course
1,2,2,1,Development,Course
2,3,3,1,Development,Course
3,4,4,1,Development,Course
4,5,5,1,Development,Course


In [17]:
arima_df = pd.DataFrame(columns=['Day_No'])
arima_df['Day_No']=np.arange(1,data.Day_No.max()+1)

df = train.groupby(['Day_No']).agg({\
                                   'Sales':{\
                                           "sale_mean":'mean',
                                            "sale_std":'std'
                                           },
                                   }).reset_index()
df.columns= ['Day_No','sale_mean','sale_std']
df.head()

model = ARIMA(df.sale_mean,order=(3,0,3))
results_arima = model.fit()
pred=results_arima.predict(start=0,end=data.Day_No.max()+1)
arima_df['day_sale_mean_arima']=round(pred,0)
#print("rmse",np.sqrt(sum((df.sale_mean-results_arima.fittedvalues)**2)))
#for sale mean arima==(3,0,0)

model = ARIMA(df.sale_std,order=(2,0,0))
results_arima = model.fit()
pred=results_arima.predict(start=0,end=data.Day_No.max()+1)
arima_df['day_sale_std_arima']=round(pred,0)
#print("rmse",np.sqrt(sum((df.sale_std-results_arima.fittedvalues)**2)))
#for sale mean arima==(2,0,0)
arima_df.head()

arima_df['Day_No']=arima_df['Day_No'].astype(int)

data = pd.merge(data,arima_df,on=['Day_No'],how='left')
data.head()

,ID,Day_No,Course_ID,Course_Domain,Course_Type,day_sale_mean_arima,day_sale_std_arima
0,1,1,1,Development,Course,121.0,45.0
1,2,2,1,Development,Course,125.0,46.0
2,3,3,1,Development,Course,119.0,46.0
3,4,4,1,Development,Course,117.0,44.0
4,5,5,1,Development,Course,116.0,42.0


In [18]:
df = train.groupby(['Day_No','Course_Domain']).agg({\
                                   'Sales':{\
                                           "sale_mean":'mean',
                                            "sale_std":'std'
                                           }
                                   }).reset_index().fillna(0)
df.columns= ['Day_No','Course_Domain','sale_mean','sale_std']

temp_df = data.copy()
temp_df = temp_df.groupby(["Day_No",'Course_Domain']).size().reset_index()
temp_df.columns=["Day_No",'Course_Domain','remove']
temp_df.drop(['remove'],axis=1,inplace=True)


vals = list(temp_df.Course_Domain.unique())
for val in vals:
    temp = pd.DataFrame(columns=['Day_No'])
    temp['Day_No']=np.arange(1,data.Day_No.max()+1)
    temp['Course_Domain']=val
    
    model = ARIMA(df[df["Course_Domain"]==val].sale_mean,order=(3,0,0))
    results_arima = model.fit()
    pred=results_arima.predict(start=0,end=data.Day_No.max())
    temp["sale_dom_mean_arima_"+str(val)]=round(pred,0)
    
    model = ARIMA(df[df["Course_Domain"]==val].sale_std,order=(3,0,0))
    results_arima = model.fit()
    pred=results_arima.predict(start=0,end=data.Day_No.max())
    temp["sale_dom_std_arima_"+str(val)]=round(pred,0)
    
    temp_df = pd.merge(temp_df,temp,on=['Day_No','Course_Domain'],how='outer')
    
temp_df['sale_dom_mean_arima_Business'].fillna(temp_df['sale_dom_mean_arima_Development'],inplace=True)
temp_df['sale_dom_mean_arima_Business'].fillna(temp_df['sale_dom_mean_arima_Finance & Accounting'],inplace=True)
temp_df['sale_dom_mean_arima_Business'].fillna(temp_df['sale_dom_mean_arima_Software Marketing'],inplace=True)

temp_df['sale_dom_std_arima_Business'].fillna(temp_df['sale_dom_std_arima_Development'],inplace=True)
temp_df['sale_dom_std_arima_Business'].fillna(temp_df['sale_dom_std_arima_Finance & Accounting'],inplace=True)
temp_df['sale_dom_std_arima_Business'].fillna(temp_df['sale_dom_std_arima_Software Marketing'],inplace=True)

temp_df.drop(['sale_dom_mean_arima_Development','sale_dom_mean_arima_Finance & Accounting','sale_dom_mean_arima_Software Marketing',\
        'sale_dom_std_arima_Development','sale_dom_std_arima_Finance & Accounting','sale_dom_std_arima_Software Marketing'],\
       axis=1,inplace=True)

temp_df.rename(columns={"sale_dom_mean_arima_Business":'sale_dom_mean_arima',\
                  "sale_dom_std_arima_Business":"sale_dom_std_arima"},inplace=True)
temp_df.head()

data = pd.merge(data,temp_df[["Day_No","Course_Domain","sale_dom_mean_arima","sale_dom_std_arima"]],on=['Day_No','Course_Domain'],how='left')
data.head()

,ID,Day_No,Course_ID,Course_Domain,Course_Type,day_sale_mean_arima,day_sale_std_arima,sale_dom_mean_arima,sale_dom_std_arima
0,1,1,1,Development,Course,121.0,45.0,120.0,49.0
1,2,2,1,Development,Course,125.0,46.0,125.0,49.0
2,3,3,1,Development,Course,119.0,46.0,120.0,49.0
3,4,4,1,Development,Course,117.0,44.0,117.0,47.0
4,5,5,1,Development,Course,116.0,42.0,115.0,46.0


In [19]:
df = train.groupby(['Day_No','Course_Type']).agg({\
                                   'Sales':{\
                                           "sale_mean":'mean',
                                            "sale_std":'std'
                                           }
                                   }).reset_index().fillna(0)
df.columns= ['Day_No','Course_Type','sale_mean','sale_std']
df.head()

temp_df = data.copy()
temp_df = temp_df.groupby(["Day_No",'Course_Type']).size().reset_index()
temp_df.columns=["Day_No",'Course_Type','remove']
temp_df.drop(['remove'],axis=1,inplace=True)

vals = list(df.Course_Type.unique())
for val in vals:
    temp = pd.DataFrame(columns=['Day_No'])
    temp['Day_No']=np.arange(1,data.Day_No.max()+1)
    temp['Course_Type']=val
    
    model = ARIMA(df[df["Course_Type"]==val].sale_mean,order=(3,0,0))
    results_arima = model.fit()
    pred=results_arima.predict(start=0,end=data.Day_No.max())
    temp["sale_ctype_mean_arima_"+str(val)]=round(pred,0)
    
    model = ARIMA(df[df["Course_Type"]==val].sale_std,order=(3,0,0))
    results_arima = model.fit()
    pred=results_arima.predict(start=0,end=data.Day_No.max())
    temp["sale_ctype_std_arima_"+str(val)]=round(pred,0)
    
    temp_df = pd.merge(temp_df,temp,on=['Day_No','Course_Type'],how='left')

    
temp_df['sale_ctype_mean_arima_Course'].fillna(temp_df['sale_ctype_mean_arima_Degree'],inplace=True)
temp_df['sale_ctype_mean_arima_Course'].fillna(temp_df['sale_ctype_mean_arima_Program'],inplace=True)

temp_df['sale_ctype_std_arima_Course'].fillna(temp_df['sale_ctype_std_arima_Degree'],inplace=True)
temp_df['sale_ctype_std_arima_Course'].fillna(temp_df['sale_ctype_std_arima_Program'],inplace=True)

temp_df.drop(['sale_ctype_mean_arima_Degree','sale_ctype_mean_arima_Program','sale_ctype_std_arima_Degree',\
        'sale_ctype_std_arima_Program'],\
       axis=1,inplace=True)

temp_df.rename(columns={"sale_ctype_mean_arima_Course":'sale_ctype_mean_arima',\
                  "sale_ctype_std_arima_Course":"sale_ctype_std_arima"},inplace=True)
temp_df.drop_duplicates(keep='first',inplace=True)
temp_df.head()

data = pd.merge(data,temp_df[["Day_No","Course_Type","sale_ctype_mean_arima","sale_ctype_std_arima"]],on=['Day_No','Course_Type'],how='left')
data.head()

,ID,Day_No,Course_ID,Course_Domain,Course_Type,day_sale_mean_arima,day_sale_std_arima,sale_dom_mean_arima,sale_dom_std_arima,sale_ctype_mean_arima,sale_ctype_std_arima
0,1,1,1,Development,Course,121.0,45.0,120.0,49.0,114.0,41.0
1,2,2,1,Development,Course,125.0,46.0,125.0,49.0,119.0,42.0
2,3,3,1,Development,Course,119.0,46.0,120.0,49.0,114.0,42.0
3,4,4,1,Development,Course,117.0,44.0,117.0,47.0,111.0,40.0
4,5,5,1,Development,Course,116.0,42.0,115.0,46.0,109.0,39.0


In [20]:
# model = ARIMA(df[df["Course_ID"]==1].sale_mean,order=(3,0,2))
# results_arima = model.fit()
# print("rmse",np.sqrt(sum((df[df["Course_ID"]==1].sale_mean-results_arima.fittedvalues)**2)))
# p=3,q=2 but for some svd wont converge so q=0 or 1
df = train.groupby(['Day_No','Course_ID']).agg({\
                                   'Sales':{\
                                           "sale_mean":'mean',
                                           }
                                   }).reset_index().fillna(0)
df.columns= ['Day_No','Course_ID','sale_mean']


temp = pd.DataFrame(columns=['Day_No'])
temp['Day_No']=np.arange(1,data.Day_No.max()+1)

vals = list(df.Course_ID.unique())
for val in vals:
    temp[str(val)]=np.nan
    
    model = ARIMA(df[df["Course_ID"]==val].sale_mean,order=(3,0,0))
    results_arima = model.fit()
    pred=results_arima.predict(start=0,end=test.Day_No.max())
    temp[str(val)]=round(pred,0)
    
t = temp.melt(id_vars=["Day_No"], 
              var_name="Course_ID",\
              value_name="sale_id_arima")
t['Day_No'] = t['Day_No'] .astype(int)
t['Course_ID'] = t['Course_ID'].astype(int)
# train = pd.merge(train,t,on=['Day_No','Course_ID'],how='left')
# test = pd.merge(test,t,on=['Day_No','Course_ID'],how='left')
data = pd.merge(data,t,on=['Day_No','Course_ID'],how='left')
data.head()

,ID,Day_No,Course_ID,Course_Domain,Course_Type,day_sale_mean_arima,day_sale_std_arima,sale_dom_mean_arima,sale_dom_std_arima,sale_ctype_mean_arima,sale_ctype_std_arima,sale_id_arima
0,1,1,1,Development,Course,121.0,45.0,120.0,49.0,114.0,41.0,85.0
1,2,2,1,Development,Course,125.0,46.0,125.0,49.0,119.0,42.0,84.0
2,3,3,1,Development,Course,119.0,46.0,120.0,49.0,114.0,42.0,83.0
3,4,4,1,Development,Course,117.0,44.0,117.0,47.0,111.0,40.0,82.0
4,5,5,1,Development,Course,116.0,42.0,115.0,46.0,109.0,39.0,84.0


In [21]:
cols = ['ID', 
       'day_sale_mean_arima', 'day_sale_std_arima', 'sale_dom_mean_arima',
       'sale_dom_std_arima', 'sale_ctype_mean_arima', 'sale_ctype_std_arima',
       'sale_id_arima']
data = data[cols]
data.head()

,ID,day_sale_mean_arima,day_sale_std_arima,sale_dom_mean_arima,sale_dom_std_arima,sale_ctype_mean_arima,sale_ctype_std_arima,sale_id_arima
0,1,121.0,45.0,120.0,49.0,114.0,41.0,85.0
1,2,125.0,46.0,125.0,49.0,119.0,42.0,84.0
2,3,119.0,46.0,120.0,49.0,114.0,42.0,83.0
3,4,117.0,44.0,117.0,47.0,111.0,40.0,82.0
4,5,116.0,42.0,115.0,46.0,109.0,39.0,84.0


In [22]:
train=pd.merge(train,data,on='ID',how='left')
test=pd.merge(test,data,on='ID',how='left')

In [23]:
train.head()

,ID,Day_No,Course_ID,Course_Domain,Course_Type,Short_Promotion,Public_Holiday,Long_Promotion,Competition_Metric,Sales,day_user_traffic_mean_arima,day_user_traffic_std_arima,user_traffic_dom_mean_arima,user_traffic_dom_std_arima,user_traffic_ctype_mean_arima,user_traffic_ctype_std_arima,user_traffic_id_arima,day_sale_mean_arima,day_sale_std_arima,sale_dom_mean_arima,sale_dom_std_arima,sale_ctype_mean_arima,sale_ctype_std_arima,sale_id_arima
0,1,1,1,Development,Course,0,1,1,1.007025,81,15398.0,7242.0,16214.0,7333.0,14990.0,6415.0,11882.0,121.0,45.0,120.0,49.0,114.0,41.0,85.0
1,2,2,1,Development,Course,0,0,1,1.007025,79,15460.0,7258.0,16098.0,7389.0,15125.0,6475.0,11617.0,125.0,46.0,125.0,49.0,119.0,42.0,84.0
2,3,3,1,Development,Course,0,0,1,1.007025,75,15454.0,7333.0,16370.0,7452.0,15170.0,6557.0,12428.0,119.0,46.0,120.0,49.0,114.0,42.0,83.0
3,4,4,1,Development,Course,0,0,1,1.007025,80,15152.0,7229.0,15934.0,7381.0,14955.0,6495.0,11798.0,117.0,44.0,117.0,47.0,111.0,40.0,82.0
4,5,5,1,Development,Course,0,0,1,1.007025,41,15261.0,6956.0,16144.0,7149.0,14958.0,6270.0,11936.0,116.0,42.0,115.0,46.0,109.0,39.0,84.0


In [24]:
test.head()

,ID,Day_No,Course_ID,Course_Domain,Course_Type,Short_Promotion,Public_Holiday,Long_Promotion,Competition_Metric,day_user_traffic_mean_arima,day_user_traffic_std_arima,user_traffic_dom_mean_arima,user_traffic_dom_std_arima,user_traffic_ctype_mean_arima,user_traffic_ctype_std_arima,user_traffic_id_arima,day_sale_mean_arima,day_sale_std_arima,sale_dom_mean_arima,sale_dom_std_arima,sale_ctype_mean_arima,sale_ctype_std_arima,sale_id_arima
0,883,883,1,Development,Course,1,0,1,1.007025,16577.0,7470.0,17016.0,7637.0,15449.0,6614.0,12577.0,145.0,49.0,139.0,54.0,128.0,45.0,100.0
1,884,884,1,Development,Course,1,0,1,1.007025,14811.0,7708.0,15911.0,7746.0,14797.0,6729.0,12075.0,123.0,53.0,125.0,57.0,120.0,48.0,95.0
2,885,885,1,Development,Course,1,0,1,1.007025,15885.0,7415.0,16921.0,7662.0,15718.0,6656.0,11935.0,132.0,48.0,131.0,54.0,124.0,45.0,85.0
3,886,886,1,Development,Course,1,0,1,1.007025,16763.0,7412.0,16502.0,7521.0,15211.0,6559.0,11897.0,140.0,48.0,126.0,53.0,119.0,44.0,84.0
4,887,887,1,Development,Course,0,0,1,1.007025,15008.0,7330.0,16230.0,7492.0,15035.0,6536.0,11886.0,120.0,47.0,123.0,52.0,117.0,43.0,84.0


In [25]:
train['Day_No'] = train["Day_No"].astype(int)
test['Day_No'] = test["Day_No"].astype(int)

train['day_sin'] = np.sin(2*np.pi*train['Day_No']/train['Day_No'].max())
test['day_sin'] = np.sin(2*np.pi*test['Day_No']/test['Day_No'].max())

In [26]:
train['total_promotion'] = train['Long_Promotion']+train['Short_Promotion']
train['shortBYLong'] = train['Short_Promotion']/(1+train['Long_Promotion'])

test['total_promotion'] = test['Long_Promotion']+test['Short_Promotion']
test['shortBYLong'] = test['Short_Promotion']/(1+test['Long_Promotion'])

In [29]:
cols = ['Course_ID', 'Course_Domain', 'Course_Type','Public_Holiday']
for col in cols:
    train[col].astype(object)
train.drop(['ID','Day_No','Course_ID'],axis=1,inplace=True)    
train = pd.get_dummies(train,columns=['Course_Domain','Course_Type'])
train.fillna(train.mean(),inplace=True)


cols = ['Course_Domain', 'Course_Type','Public_Holiday']
for col in cols:
    test[col].astype(object)
test.index = test.ID
test.drop(['ID','Day_No','Course_ID'],axis=1,inplace=True)    
test = pd.get_dummies(test,columns=['Course_Domain', 'Course_Type'])
test.fillna(test.mean(),inplace=True)

In [30]:
test.shape

(36000, 28)

In [31]:
train.shape

(512087, 29)

In [32]:
train.head()

,Short_Promotion,Public_Holiday,Long_Promotion,Competition_Metric,Sales,day_user_traffic_mean_arima,day_user_traffic_std_arima,user_traffic_dom_mean_arima,user_traffic_dom_std_arima,user_traffic_ctype_mean_arima,user_traffic_ctype_std_arima,user_traffic_id_arima,day_sale_mean_arima,day_sale_std_arima,sale_dom_mean_arima,sale_dom_std_arima,sale_ctype_mean_arima,sale_ctype_std_arima,sale_id_arima,day_sin,total_promotion,shortBYLong,Course_Domain_Business,Course_Domain_Development,Course_Domain_Finance & Accounting,Course_Domain_Software Marketing,Course_Type_Course,Course_Type_Degree,Course_Type_Program
0,0,1,1,1.007025,81,15398.0,7242.0,16214.0,7333.0,14990.0,6415.0,11882.0,121.0,45.0,120.0,49.0,114.0,41.0,85.0,0.007124,1,0.0,0,1,0,0,1,0,0
1,0,0,1,1.007025,79,15460.0,7258.0,16098.0,7389.0,15125.0,6475.0,11617.0,125.0,46.0,125.0,49.0,119.0,42.0,84.0,0.014247,1,0.0,0,1,0,0,1,0,0
2,0,0,1,1.007025,75,15454.0,7333.0,16370.0,7452.0,15170.0,6557.0,12428.0,119.0,46.0,120.0,49.0,114.0,42.0,83.0,0.021370,1,0.0,0,1,0,0,1,0,0
3,0,0,1,1.007025,80,15152.0,7229.0,15934.0,7381.0,14955.0,6495.0,11798.0,117.0,44.0,117.0,47.0,111.0,40.0,82.0,0.028491,1,0.0,0,1,0,0,1,0,0
4,0,0,1,1.007025,41,15261.0,6956.0,16144.0,7149.0,14958.0,6270.0,11936.0,116.0,42.0,115.0,46.0,109.0,39.0,84.0,0.035611,1,0.0,0,1,0,0,1,0,0


In [33]:
test.head()

,Short_Promotion,Public_Holiday,Long_Promotion,Competition_Metric,day_user_traffic_mean_arima,day_user_traffic_std_arima,user_traffic_dom_mean_arima,user_traffic_dom_std_arima,user_traffic_ctype_mean_arima,user_traffic_ctype_std_arima,user_traffic_id_arima,day_sale_mean_arima,day_sale_std_arima,sale_dom_mean_arima,sale_dom_std_arima,sale_ctype_mean_arima,sale_ctype_std_arima,sale_id_arima,day_sin,total_promotion,shortBYLong,Course_Domain_Business,Course_Domain_Development,Course_Domain_Finance & Accounting,Course_Domain_Software Marketing,Course_Type_Course,Course_Type_Degree,Course_Type_Program
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,
883,1,0,1,1.007025,16577.0,7470.0,17016.0,7637.0,15449.0,6614.0,12577.0,145.0,49.0,139.0,54.0,128.0,45.0,100.0,-0.383454,2,0.5,0,1,0,0,1,0,0
884,1,0,1,1.007025,14811.0,7708.0,15911.0,7746.0,14797.0,6729.0,12075.0,123.0,53.0,125.0,57.0,120.0,48.0,95.0,-0.377285,2,0.5,0,1,0,0,1,0,0
885,1,0,1,1.007025,15885.0,7415.0,16921.0,7662.0,15718.0,6656.0,11935.0,132.0,48.0,131.0,54.0,124.0,45.0,85.0,-0.371099,2,0.5,0,1,0,0,1,0,0
886,1,0,1,1.007025,16763.0,7412.0,16502.0,7521.0,15211.0,6559.0,11897.0,140.0,48.0,126.0,53.0,119.0,44.0,84.0,-0.364897,2,0.5,0,1,0,0,1,0,0
887,0,0,1,1.007025,15008.0,7330.0,16230.0,7492.0,15035.0,6536.0,11886.0,120.0,47.0,123.0,52.0,117.0,43.0,84.0,-0.358679,1,0.0,0,1,0,0,1,0,0


In [82]:
train['daytraffic__comp'] = train['day_user_traffic_mean_arima']/train['Competition_Metric']
test['daytraffic__comp'] = test['day_user_traffic_mean_arima']/test['Competition_Metric']

In [85]:
train.columns

Index(['Short_Promotion', 'Public_Holiday', 'Long_Promotion',
       'Competition_Metric', 'Sales', 'day_user_traffic_mean_arima',
       'day_user_traffic_std_arima', 'user_traffic_dom_mean_arima',
       'user_traffic_dom_std_arima', 'user_traffic_ctype_mean_arima',
       'user_traffic_ctype_std_arima', 'user_traffic_id_arima',
       'day_sale_mean_arima', 'day_sale_std_arima', 'sale_dom_mean_arima',
       'sale_dom_std_arima', 'sale_ctype_mean_arima', 'sale_ctype_std_arima',
       'sale_id_arima', 'day_sin', 'total_promotion', 'shortBYLong',
       'Course_Domain_Business', 'Course_Domain_Development',
       'Course_Domain_Finance & Accounting',
       'Course_Domain_Software Marketing', 'Course_Type_Course',
       'Course_Type_Degree', 'Course_Type_Program', 'daytraffic__comp'],
      dtype='object')

In [86]:
X = train[[col for col in train.columns if col not in (['Sales','day_user_traffic_std_arima', \
                                                        'user_traffic_dom_std_arima','user_traffic_ctype_std_arima'])]]
#test.drop([],axis=1,inplace=True)
y = train.Sales

kf = KFold(n_splits=10)
test_preds = np.zeros((len(test),1))

lr = LinearRegression()
rfc = RandomForestRegressor()
lgbm = lightgbm.LGBMRegressor(n_estimators=1000,\
                             max_depth=8,\
                             #colsample_bytree=0.9
                             )

cols_name = ['lgbm','rfc','lr']
result = pd.DataFrame(columns=cols_name)
feature_imp = np.zeros((len(X.columns),1))
i=0
classfiers = [lgbm,rfc,lr]
for clf in classfiers:
    test_preds = np.zeros((len(test),1))
    scores = []
    for train_idx,val_idx in kf.split(X,y):
        X_train,X_val = X.iloc[train_idx],X.iloc[val_idx]
        y_train,y_val = y.iloc[train_idx],y.iloc[val_idx]

        clf.fit(X_train,y_train)
        pred = clf.predict(X_val)
        #feature_imp += clf.feature_importances_.reshape(-1,1)
        pred = np.where(pred<0,0,pred)
        score = np.sqrt(mean_squared_log_error(y_val,pred))
        scores.append(score)
        print(1000*score)

        test_pred = clf.predict(test).reshape(-1,1)
        test_pred = np.where(test_pred<0,0,test_pred)
        test_preds += test_pred
        
    test_preds = test_preds/10
    result.iloc[:,i]=test_preds[:,0]
    print(clf)
    print(round(np.mean(scores),2),end="\n")
    i+=1
    

# feature_imp = feature_imp/10
# print("\nmean score= ",1000*np.mean(scores))

# feature_imp = pd.DataFrame(feature_imp, columns=['Value'])
# feature_imp['columns']=X.columns
# feature_imp = feature_imp.sort_values(by='Value')

155.89580011666112
159.29768216536783
163.85417154770175
157.6774263882008
181.53699600002003
169.93723652861152
164.27636647382866
163.13575885088127
154.8070374939765
151.44006842929218
LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=8,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=1000, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
0.16


In [95]:
test_nn = test.copy()
cols = [\
       'day_sale_mean_arima', 'day_sale_std_arima', 'sale_dom_mean_arima',
       'sale_dom_std_arima', 'sale_ctype_mean_arima', 'sale_ctype_std_arima',
       'sale_id_arima']
X_nn = X[cols]

test_nn = test_nn[cols]

from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.wrappers.scikit_learn import KerasRegressor



clf = Sequential([
                    Dense(32, activation='relu', input_shape=[len(X_nn.columns)]),
                    Dense(64, activation='relu'),
                    Dense(96, activation='relu'),
                    Dense(32, activation='relu'),
                    Dense(1, activation='linear')
                  ])
clf.compile(loss='mse',optimizer='adam',metrics=[tf.keras.losses.msle])

history = clf.fit(X_nn,y,validation_split=0.1,epochs=300,batch_size=1000,verbose=2)



Train on 460878 samples, validate on 51209 samples
Epoch 1/500
 - 1s - loss: 1461.9998 - mean_squared_logarithmic_error: 0.2322 - val_loss: 1023.3608 - val_mean_squared_logarithmic_error: 0.0943
Epoch 2/500
 - 1s - loss: 986.9964 - mean_squared_logarithmic_error: 0.0964 - val_loss: 1034.8355 - val_mean_squared_logarithmic_error: 0.0906
Epoch 3/500
 - 1s - loss: 974.7195 - mean_squared_logarithmic_error: 0.0956 - val_loss: 1002.2078 - val_mean_squared_logarithmic_error: 0.0957
Epoch 4/500
 - 1s - loss: 965.9398 - mean_squared_logarithmic_error: 0.0951 - val_loss: 973.9045 - val_mean_squared_logarithmic_error: 0.0911
Epoch 5/500
 - 1s - loss: 955.8814 - mean_squared_logarithmic_error: 0.0945 - val_loss: 959.9298 - val_mean_squared_logarithmic_error: 0.0905
Epoch 6/500
 - 1s - loss: 938.5569 - mean_squared_logarithmic_error: 0.0935 - val_loss: 952.0939 - val_mean_squared_logarithmic_error: 0.0917
Epoch 7/500
 - 1s - loss: 924.6380 - mean_squared_logarithmic_error: 0.0927 - val_loss: 942.4

Epoch 58/500
 - 1s - loss: 643.4301 - mean_squared_logarithmic_error: 0.0621 - val_loss: 718.2423 - val_mean_squared_logarithmic_error: 0.0647
Epoch 59/500
 - 1s - loss: 642.3087 - mean_squared_logarithmic_error: 0.0619 - val_loss: 686.3661 - val_mean_squared_logarithmic_error: 0.0628
Epoch 60/500
 - 1s - loss: 641.1714 - mean_squared_logarithmic_error: 0.0620 - val_loss: 675.4466 - val_mean_squared_logarithmic_error: 0.0592
Epoch 61/500
 - 1s - loss: 638.8375 - mean_squared_logarithmic_error: 0.0618 - val_loss: 681.1463 - val_mean_squared_logarithmic_error: 0.0603
Epoch 62/500
 - 1s - loss: 635.7738 - mean_squared_logarithmic_error: 0.0616 - val_loss: 661.6923 - val_mean_squared_logarithmic_error: 0.0588
Epoch 63/500
 - 1s - loss: 636.1612 - mean_squared_logarithmic_error: 0.0615 - val_loss: 673.1985 - val_mean_squared_logarithmic_error: 0.0610
Epoch 64/500
 - 1s - loss: 632.5171 - mean_squared_logarithmic_error: 0.0611 - val_loss: 677.0639 - val_mean_squared_logarithmic_error: 0.0612

Epoch 116/500
 - 1s - loss: 555.0507 - mean_squared_logarithmic_error: 0.0535 - val_loss: 608.9449 - val_mean_squared_logarithmic_error: 0.0600
Epoch 117/500
 - 1s - loss: 550.3145 - mean_squared_logarithmic_error: 0.0530 - val_loss: 592.8452 - val_mean_squared_logarithmic_error: 0.0549
Epoch 118/500
 - 1s - loss: 551.1986 - mean_squared_logarithmic_error: 0.0530 - val_loss: 575.1737 - val_mean_squared_logarithmic_error: 0.0514
Epoch 119/500
 - 1s - loss: 552.6160 - mean_squared_logarithmic_error: 0.0534 - val_loss: 577.7119 - val_mean_squared_logarithmic_error: 0.0511
Epoch 120/500
 - 1s - loss: 547.7720 - mean_squared_logarithmic_error: 0.0531 - val_loss: 575.8257 - val_mean_squared_logarithmic_error: 0.0492
Epoch 121/500
 - 1s - loss: 547.7683 - mean_squared_logarithmic_error: 0.0532 - val_loss: 567.1476 - val_mean_squared_logarithmic_error: 0.0500
Epoch 122/500
 - 1s - loss: 547.7273 - mean_squared_logarithmic_error: 0.0526 - val_loss: 576.9083 - val_mean_squared_logarithmic_error:

Epoch 173/500
 - 1s - loss: 492.9595 - mean_squared_logarithmic_error: 0.0479 - val_loss: 525.4199 - val_mean_squared_logarithmic_error: 0.0461
Epoch 174/500
 - 1s - loss: 492.5687 - mean_squared_logarithmic_error: 0.0475 - val_loss: 521.4714 - val_mean_squared_logarithmic_error: 0.0464
Epoch 175/500
 - 1s - loss: 492.6864 - mean_squared_logarithmic_error: 0.0481 - val_loss: 517.5203 - val_mean_squared_logarithmic_error: 0.0446
Epoch 176/500
 - 2s - loss: 493.5360 - mean_squared_logarithmic_error: 0.0483 - val_loss: 505.8354 - val_mean_squared_logarithmic_error: 0.0464
Epoch 177/500
 - 2s - loss: 490.3108 - mean_squared_logarithmic_error: 0.0479 - val_loss: 513.0279 - val_mean_squared_logarithmic_error: 0.0450
Epoch 178/500
 - 1s - loss: 490.5510 - mean_squared_logarithmic_error: 0.0481 - val_loss: 531.7763 - val_mean_squared_logarithmic_error: 0.0542
Epoch 179/500
 - 1s - loss: 494.0295 - mean_squared_logarithmic_error: 0.0485 - val_loss: 548.5129 - val_mean_squared_logarithmic_error:

Epoch 230/500
 - 1s - loss: 460.9635 - mean_squared_logarithmic_error: 0.0452 - val_loss: 477.4072 - val_mean_squared_logarithmic_error: 0.0460
Epoch 231/500
 - 1s - loss: 457.7128 - mean_squared_logarithmic_error: 0.0445 - val_loss: 489.1168 - val_mean_squared_logarithmic_error: 0.0460
Epoch 232/500
 - 1s - loss: 461.2370 - mean_squared_logarithmic_error: 0.0454 - val_loss: 475.1641 - val_mean_squared_logarithmic_error: 0.0400
Epoch 233/500
 - 1s - loss: 458.6488 - mean_squared_logarithmic_error: 0.0445 - val_loss: 488.8325 - val_mean_squared_logarithmic_error: 0.0443
Epoch 234/500
 - 1s - loss: 458.8385 - mean_squared_logarithmic_error: 0.0450 - val_loss: 483.9679 - val_mean_squared_logarithmic_error: 0.0446
Epoch 235/500
 - 1s - loss: 462.1297 - mean_squared_logarithmic_error: 0.0455 - val_loss: 479.0239 - val_mean_squared_logarithmic_error: 0.0405
Epoch 236/500
 - 1s - loss: 460.0541 - mean_squared_logarithmic_error: 0.0453 - val_loss: 516.3290 - val_mean_squared_logarithmic_error:

Epoch 287/500
 - 1s - loss: 449.0674 - mean_squared_logarithmic_error: 0.0443 - val_loss: 451.9113 - val_mean_squared_logarithmic_error: 0.0399
Epoch 288/500
 - 1s - loss: 443.4081 - mean_squared_logarithmic_error: 0.0429 - val_loss: 452.5748 - val_mean_squared_logarithmic_error: 0.0403
Epoch 289/500
 - 1s - loss: 444.9390 - mean_squared_logarithmic_error: 0.0441 - val_loss: 457.2792 - val_mean_squared_logarithmic_error: 0.0408
Epoch 290/500
 - 1s - loss: 441.9539 - mean_squared_logarithmic_error: 0.0436 - val_loss: 443.0871 - val_mean_squared_logarithmic_error: 0.0408
Epoch 291/500
 - 1s - loss: 441.8749 - mean_squared_logarithmic_error: 0.0434 - val_loss: 457.9797 - val_mean_squared_logarithmic_error: 0.0385
Epoch 292/500
 - 1s - loss: 442.1649 - mean_squared_logarithmic_error: 0.0434 - val_loss: 461.7126 - val_mean_squared_logarithmic_error: 0.0403
Epoch 293/500
 - 1s - loss: 439.8654 - mean_squared_logarithmic_error: 0.0429 - val_loss: 448.5704 - val_mean_squared_logarithmic_error:

Epoch 344/500
 - 1s - loss: 431.4434 - mean_squared_logarithmic_error: 0.0427 - val_loss: 445.0703 - val_mean_squared_logarithmic_error: 0.0399
Epoch 345/500
 - 1s - loss: 433.4833 - mean_squared_logarithmic_error: 0.0425 - val_loss: 453.5130 - val_mean_squared_logarithmic_error: 0.0399
Epoch 346/500
 - 1s - loss: 429.2656 - mean_squared_logarithmic_error: 0.0424 - val_loss: 435.7429 - val_mean_squared_logarithmic_error: 0.0371
Epoch 347/500
 - 1s - loss: 428.5049 - mean_squared_logarithmic_error: 0.0417 - val_loss: 446.0868 - val_mean_squared_logarithmic_error: 0.0378
Epoch 348/500
 - 1s - loss: 433.6684 - mean_squared_logarithmic_error: 0.0433 - val_loss: 444.7079 - val_mean_squared_logarithmic_error: 0.0423
Epoch 349/500
 - 1s - loss: 428.1918 - mean_squared_logarithmic_error: 0.0422 - val_loss: 439.7978 - val_mean_squared_logarithmic_error: 0.0396
Epoch 350/500
 - 1s - loss: 433.6072 - mean_squared_logarithmic_error: 0.0428 - val_loss: 475.1652 - val_mean_squared_logarithmic_error:

Epoch 401/500
 - 1s - loss: 424.3808 - mean_squared_logarithmic_error: 0.0418 - val_loss: 435.5955 - val_mean_squared_logarithmic_error: 0.0381
Epoch 402/500
 - 1s - loss: 424.1748 - mean_squared_logarithmic_error: 0.0420 - val_loss: 440.8152 - val_mean_squared_logarithmic_error: 0.0432
Epoch 403/500
 - 1s - loss: 426.2539 - mean_squared_logarithmic_error: 0.0423 - val_loss: 442.6226 - val_mean_squared_logarithmic_error: 0.0427
Epoch 404/500
 - 1s - loss: 424.2682 - mean_squared_logarithmic_error: 0.0415 - val_loss: 429.2352 - val_mean_squared_logarithmic_error: 0.0371
Epoch 405/500
 - 1s - loss: 425.6922 - mean_squared_logarithmic_error: 0.0418 - val_loss: 443.6361 - val_mean_squared_logarithmic_error: 0.0401
Epoch 406/500
 - 1s - loss: 425.3302 - mean_squared_logarithmic_error: 0.0421 - val_loss: 432.3195 - val_mean_squared_logarithmic_error: 0.0409
Epoch 407/500
 - 1s - loss: 427.2818 - mean_squared_logarithmic_error: 0.0422 - val_loss: 436.0083 - val_mean_squared_logarithmic_error:

Epoch 458/500
 - 2s - loss: 421.7591 - mean_squared_logarithmic_error: 0.0421 - val_loss: 427.2659 - val_mean_squared_logarithmic_error: 0.0373
Epoch 459/500
 - 2s - loss: 417.8356 - mean_squared_logarithmic_error: 0.0410 - val_loss: 421.8539 - val_mean_squared_logarithmic_error: 0.0369
Epoch 460/500
 - 2s - loss: 424.1954 - mean_squared_logarithmic_error: 0.0421 - val_loss: 426.4352 - val_mean_squared_logarithmic_error: 0.0370
Epoch 461/500
 - 2s - loss: 419.2238 - mean_squared_logarithmic_error: 0.0409 - val_loss: 437.0892 - val_mean_squared_logarithmic_error: 0.0401
Epoch 462/500
 - 2s - loss: 416.8346 - mean_squared_logarithmic_error: 0.0410 - val_loss: 414.1154 - val_mean_squared_logarithmic_error: 0.0400
Epoch 463/500
 - 2s - loss: 417.8719 - mean_squared_logarithmic_error: 0.0415 - val_loss: 427.7153 - val_mean_squared_logarithmic_error: 0.0408
Epoch 464/500
 - 2s - loss: 418.6339 - mean_squared_logarithmic_error: 0.0413 - val_loss: 456.5488 - val_mean_squared_logarithmic_error:

In [96]:
test_pred = clf.predict(test_nn)
result['keras'] = test_pred

In [ ]:
result['Sales'] = result.sum(axis=1)/4

In [76]:
temp = test.copy()
temp['Sales'] = result.Sales.values
temp = temp[['Sales']]
temp.to_csv('try11.csv')
temp.head()

,Sales
ID,
883,121.184773
884,114.229258
885,93.374565
886,103.552862
887,80.850657
